In [ ]:
# Copyright 2025 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Intro to Vertex AI Global Endpoint


<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/gemini/global-endpoint/intro_global_endpoint.ipynb">
      <img width="32px" src="https://www.gstatic.com/pantheon/images/bigquery/welcome_page/colab-logo.svg" alt="Google Colaboratory logo"><br> Open in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fgenerative-ai%2Fmain%2Fgemini%2Fglobal-endpoint%2Fintro_global_endpoint.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> Open in Colab Enterprise
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/main/gemini/global-endpoint/intro_global_endpoint.ipynb">
      <img src="https://www.gstatic.com/images/branding/gcpiconscolors/vertexai/v1/32px.svg" alt="Vertex AI logo"><br> Open in Vertex AI Workbench
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/global-endpoint/intro_global_endpoint.ipynb">
      <img width="32px" src="https://www.svgrepo.com/download/217753/github.svg" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
</table>

<div style="clear: both;"></div>

<b>Share to:</b>

<a href="https://www.linkedin.com/sharing/share-offsite/?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/global-endpoint/intro_global_endpoint.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/8/81/LinkedIn_icon.svg" alt="LinkedIn logo">
</a>

<a href="https://bsky.app/intent/compose?text=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/global-endpoint/intro_global_endpoint.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/7/7a/Bluesky_Logo.svg" alt="Bluesky logo">
</a>

<a href="https://twitter.com/intent/tweet?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/global-endpoint/intro_global_endpoint.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/5/5a/X_icon_2.svg" alt="X logo">
</a>

<a href="https://reddit.com/submit?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/global-endpoint/intro_global_endpoint.ipynb" target="_blank">
  <img width="20px" src="https://redditinc.com/hubfs/Reddit%20Inc/Brand/Reddit_Logo.png" alt="Reddit logo">
</a>

<a href="https://www.facebook.com/sharer/sharer.php?u=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/global-endpoint/intro_global_endpoint.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/5/51/Facebook_f_logo_%282019%29.svg" alt="Facebook logo">
</a>

| | |
|-|-|
| Author(s) |  [Eric Dong](https://github.com/gericdong) |

## Overview

Vertex AI global endpoint enables developers to leverage Google's highly scalable and global infrastructure for Gemini models. Leveraging the global endpoint can significantly improve overall service availability and throughput, while reducing the likelihood of rate limiting (429 RESOURCE_EXHAUSTED errors).

This tutorial demonstrates how to call the Vertex AI global endpoint using both the REST API and the Google Gen AI SDK.

**Important**:

- Use the global endpoint if you don't have strict regional restriction requirements.
- The global endpoint supports the stable model versions for generally available Gemini models such as `gemini-2.0-flash-001` and `gemini-2.0-flash-lite-001`.

## Getting Started

### Install Google Gen AI SDK


In [ ]:
%pip install --upgrade --quiet google-genai

### Authenticate your notebook environment

If you are running this notebook on Google Colab, run the cell below to authenticate your environment.

In [ ]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

### Set Google Cloud project ID

To start using Vertex AI, you must have an existing Google Cloud project and [enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com).

Learn more about [setting up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).

In [ ]:
import os

PROJECT_ID = (
    "[your-project-id]"  # @param {type: "string", placeholder: "[your-project-id]"}
)
if not PROJECT_ID or PROJECT_ID == "[your-project-id]":
    PROJECT_ID = str(os.environ.get("GOOGLE_CLOUD_PROJECT"))

### Use the Gemini 2.0 Flash model

The global endpoint currently supports the following Gemini 2 models:

- Gemini 2.0 Flash Lite 001 (`gemini-2.0-flash-lite-001`)
- Gemini 2.0 Flash 001 (`gemini-2.0-flash-001`)


In [ ]:
MODEL_ID = "gemini-2.0-flash-001"  # @param {type: "string"}

## Use the Global Endpoint

You set location to `global` to specify a global endpoint.

In [ ]:
LOCATION = "global"  # @param {type: "string"}

### **Option 1**: Use global endpoint with REST API

Next, send a REST API request to the Gemini model using cURL with the global endpoint. When forming the request URL for the global endpoint, ensure the hostname `API_HOST` does not include a region identifier (unlike regional endpoints), and specify `global` within the URL path where the location would normally appear.


In [ ]:
API_HOST = "aiplatform.googleapis.com"

os.environ["API_ENDPOINT"] = (
    f"{API_HOST}/v1/projects/{PROJECT_ID}/locations/{LOCATION}/publishers/google/models/{MODEL_ID}"
)

The following example shows a text generation that uses a global endpoint.

In [ ]:
%%bash

curl -X POST \
  -H "Authorization: Bearer $(gcloud auth print-access-token)" \
  -H "Content-Type: application/json" \
  https://${API_ENDPOINT}:generateContent \
  -d '{
    "contents": {
      "role": "USER",
      "parts": { "text": "Why is the sky blue?" },
    },
  }'

### **Option 2**: Use global endpoint with Gen AI SDK

When you use the Gen AI SDK, set the location as part of the client options. Here you create a `client` for Vertex AI service, and set  location `global` to indicate to use a global endpoint.

In [ ]:
from google import genai

client = genai.Client(vertexai=True, project=PROJECT_ID, location=LOCATION)

The following example shows a text generation that uses a global endpoint.

In [ ]:
response = client.models.generate_content(
    model=MODEL_ID, contents="Why is the sky blue?"
)

print(response.text)

## What's next

- Learn more about [Generative AI on Vertex AI locations](https://cloud.google.com/vertex-ai/generative-ai/docs/learn/locations).
- Learn more about [Model versions and lifecycle](https://cloud.google.com/vertex-ai/generative-ai/docs/learn/model-versions).
- Explore other notebooks in the [Google Cloud Generative AI  repository](https://github.com/GoogleCloudPlatform/generative-ai).